In [1]:
using Revise, NeuralNetworkAnalysis, NeuralVerification, YAML
const NV = NeuralVerification
const RA = ReachabilityAnalysis
const IA = IntervalArithmetic
const NNA = NeuralNetworkAnalysis

┌ Warning: Replacing docs for `LazySets.HPolytope :: Union{Tuple{Union{Polyhedra.HRepresentation{N}, Polyhedra.Polyhedron{N}}}, Tuple{N}} where N` in module `LazySets`
└ @ Base.Docs docs/Docs.jl:227
┌ Warning: Replacing docs for `LazySets.HPolyhedron :: Union{Tuple{Union{Polyhedra.HRepresentation{N}, Polyhedra.Polyhedron{N}}}, Tuple{N}} where N` in module `LazySets`
└ @ Base.Docs docs/Docs.jl:227
┌ Warning: Replacing docs for `Polyhedra.polyhedron :: Union{Tuple{Union{HPolyhedron{N,VN} where VN<:AbstractArray{N,1}, HPolytope{N,VN} where VN<:AbstractArray{N,1}}}, Tuple{N}} where N<:Real` in module `LazySets`
└ @ Base.Docs docs/Docs.jl:227
┌ Warning: Replacing docs for `Polyhedra.polyhedron :: Union{Tuple{VPolytope{N,VN} where VN<:AbstractArray{N,1}}, Tuple{N}} where N<:Real` in module `LazySets`
└ @ Base.Docs docs/Docs.jl:227


NeuralNetworkAnalysis

In [2]:
d = YAML.load_file("sig16x16.yml")
nnet = NNA.read_yaml(d);

In [3]:
X0 = Hyperrectangle(low=[-0.6, 0], high=[-0.4, 0])
X0s = split(X0, [1, 1])
sol = [NNA.forward(nnet, X0) for X0 in X0s];

In [4]:
overapproximate(UnionSetArray(sol), Interval) # 8

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.783394, -0.544671])

In [5]:
overapproximate(UnionSetArray(sol), Interval) # 4

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.783394, -0.544671])

In [6]:
overapproximate(UnionSetArray(sol), Interval) # 1

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.783394, -0.544671])

In [7]:
function step(p, v, u)
    return (p + v, v + 0.0015u - 0.0025 * cos(3p))
end

step (generic function with 1 method)

In [76]:
p, v = -0.5..0.501, 0. .. 0.

([-0.5, 0.501001], [0, 0])

In [83]:
function mountain_car(p, v, nnet; max_setps=150)
    reward = 100.
    n = 0
    for i=1:max_setps
        n += 1
        u = forward(nnet, Interval(p) × Interval(v))
        u = u.array[1].dat
        reward = reward - 0.1 * u^2
        p, v = step(p, v, u)
        if p.lo >= 0.45
            break
        end
        IJulia.clear_output(true)
        println("reward: " * string(reward))
        println("step: " * string(i))
        println("p.hi:" * string(p.hi))
        println("p.lo:" * string(p.lo))
    end
    return p, v, reward, n
end

mountain_car (generic function with 1 method)

In [84]:
@time sol = mountain_car(p, v, nnet);

reward: [90.2169, 100]
step: 10
p.hi:0.6001062069187053
p.lo:-0.7090946036460517


LoadError: InterruptException:

In [11]:
@taylorize function car!(dx, x, p, t)
    p, v, u = x
    dx[1] = p + v
    dx[2] = v + 0.0015u - 0.0025 * cos(3p)
    dx[3] = zero(u)

    return dx
end

X₀ = Hyperrectangle(low=[-0.41, 0], high=[-0.4, 0])
U₀ = Universe(1)
prob = @ivp(x' = car!(x), dim: 3, x(0) ∈ X₀ × U₀)
vars_idx = Dict(:state_vars=>1:2, :input_vars=>[], :control_vars=>[3]);

In [12]:
plant = ControlledPlant(prob, nnet, vars_idx);
alg = TMJets(abs_tol=1e-9, orderT=8, orderQ=1)
solver = "hybrid"

@time sol = NNA.solve(plant, T=0.6, Tsample=0.2, alg_nn=solver, alg=alg)
#solz = overapproximate(sol, Zonotope);

LoadError: MethodError: no method matching _solve(::ControlledPlant{BlackBoxContinuousSystem{typeof(car!)},CartesianProduct{Float64,Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}},Universe{Float64}},InitialValueProblem{BlackBoxContinuousSystem{typeof(car!)},CartesianProduct{Float64,Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}},Universe{Float64}}},AbstractArray{T,1} where T}, ::TMJets{Float64,ZonotopeEnclosure}, ::String, ::IntervalArithmetic.Interval{Float64}, ::Float64, ::Bool, ::NeuralNetworkAnalysis.var"#6#7")
Closest candidates are:
  _solve(::ControlledPlant, ::ReachabilityAnalysis.AbstractContinuousPost, !Matched::Solver, ::IntervalArithmetic.Interval{Float64}, ::N, ::Bool, ::Function) where N at /home/sguadalupe/.julia/dev/NeuralNetworkAnalysis/src/solve.jl:137

In [13]:
sol

1-element Array{CartesianProductArray{Float64,Interval{Float64,IntervalArithmetic.Interval{Float64}}},1}:
 CartesianProductArray{Float64,Interval{Float64,IntervalArithmetic.Interval{Float64}}}(Interval{Float64,IntervalArithmetic.Interval{Float64}}[Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.783394, -0.544671])])